In [1]:
%config IPCompleter.greedy=True

In [64]:
from pyspark import SparkContext, RDD
from string import punctuation
import findspark as fs
import random

In [3]:
fs.init()
sc = SparkContext(appName="NumbersAndRandomParagraph")

# Numbers

In [8]:
# generating file with numbers
with open('./data/numbers.txt', 'w') as f:
    for _ in range(0, 10000):
        next = random.randint(0, 10000)
        f.write('{number}\n'.format(number=str(next)))

In [23]:
# open file with Spark
file = sc.textFile('./data/numbers.txt')

In [32]:
numbers = file.map(lambda x: int(x))

In [45]:
def min_callback(acc, cur):
    if cur < acc:
        acc = cur
        
    return acc

def max_callback(acc, cur):
    if cur > acc:
        acc = cur
        
    return acc

In [58]:
# min and max
min = numbers.fold(numbers.first(), min_callback)
max = numbers.fold(numbers.first(), max_callback)
print('Min:', min)
print('Max:', max)

Min: 1
Max: 10000


In [59]:
# mean
mean = numbers.reduce(lambda acc, cur: acc + cur) / numbers.count()
print('Mean:', mean)

Mean: 5016.275


In [61]:
# distinct set of numbers
distinct_set = numbers.map(lambda x: (x, 1))
distinct_set = distinct_set.reduceByKey(lambda acc, _: acc)
distinct_set = distinct_set.map(lambda x: x[0])

distinct_set.saveAsTextFile("results_distinct_set")

In [63]:
# number of different elements without repetition
print('Number of different elements without repetition:', distinct_set.count())

Number of different elements without repetition: 6341


# Random paragraph

In [65]:
text = sc.textFile('../data/cobc.txt')

In [67]:
words = text.flatMap(lambda line: line.split(' '))
words = words.map(lambda word: word.lower().translate(str.maketrans('', '', punctuation)))
words = words.filter(lambda word: word not in stopwords and word and not word.isdigit())

In [68]:
successors = words.map(lambda word: (word, []))

In [71]:
part = words.partitionBy(2)
part.glom().collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 65.0 failed 1 times, most recent failure: Lost task 0.0 in stage 65.0 (TID 112, 192.168.1.112, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/home/jakub/.local/lib/python3.8/site-packages/pyspark/rdd.py", line 1856, in add_shuffle_key
    for k, v in iterator:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 107, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-67-1a38e57af968>